# Importing the Libraries


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

C:\Users\ashut\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Datasets


In [3]:
# Importing the Philadelphia buildings dataset

df1 = pd.read_csv('phl_buildings.csv')
df1.head()

,OBJECTID,BIN,FCODE,ADDRESS,BUILDING_NAME,BASE_ELEVATION,APPROX_HGT,MAX_HGT,PARCEL_ID_NUM,PARCEL_ID_SOURCE,Shape__Area,Shape__Length
0,51065281,1000001,1810,1501 N 18TH ST,NaN,104.2,20.0,28.0,319860,PWD,97.761719,40.383631
1,51065282,1000002,1810,1501 N 18TH ST,NaN,103.3,20.0,29.0,319860,PWD,99.632812,40.691884
2,51065283,1000003,1810,1501 N 18TH ST,NaN,103.9,20.0,28.0,319860,PWD,96.292969,40.145485
3,51065284,1000004,1810,1501 N 18TH ST,NaN,103.6,20.0,28.0,319860,PWD,98.699219,40.543015
4,51065285,1000005,1810,1501 N 18TH ST,NaN,108.0,20.0,30.0,319860,PWD,101.902344,42.790290


In [4]:
print(f"No. of rows in df1:", len(df1))
print(f"No. columns in df1:", len(df1.columns))

No. of rows in df1: 543278
No. columns in df1: 12


In [5]:
# Importing the Philadelphia Properties dataset
df2 = pd.read_csv("phl_properties.csv", low_memory = False)

print(f"No. of rows in df2:", len(df2))
print(f"No. columns in df2:", len(df2.columns))

No. of rows in df2: 581456
No. columns in df2: 75


<b> Since we are interested in in property prie prediction. That's why we are nly going to use the Philadelphia Properties dataset. </b>


In [6]:
df = df2.copy()

In [7]:
# Removing the columns which do not contribute to the property price

df.drop(columns=['assessment_date','beginning_point','book_and_page','census_tract','cross_reference','date_exterior_condition',
                 'exempt_building','exempt_land','geographic_ward','homestead_exemption','house_extension','house_number',
                 'mailing_zip','market_value_date','off_street_open','separate_utilities','sewer','site_type','state_code',
                 'street_code','street_direction','street_name','suffix','taxable_building','taxable_land','utility',
                 'year_built_estimate','objectid','mailing_address_1','mailing_address_2','mailing_care_of','mailing_city_state',
                 ], inplace=True)

In [8]:
df.head()

,basements,building_code,building_code_description,category_code,category_code_description,central_air,depth,exterior_condition,fireplaces,frontage,...,topography,total_area,total_livable_area,type_heater,unfinished,unit,view_type,year_built,zip_code,zoning
0,D,P50,ROW W/GAR 3 STY MASONRY,1,Single Family,N,49.0,5.0,0.0,15.90,...,F,779.0,1650.0,H,NaN,NaN,I,1920,191475425,RSA5
1,D,O50,ROW 3 STY MASONRY,1,Single Family,Y,49.0,2.0,0.0,15.90,...,F,779.1,1203.0,H,NaN,NaN,I,1920,191475425,RSA5
2,D,O50,ROW 3 STY MASONRY,1,Single Family,N,49.0,4.0,0.0,14.80,...,F,725.2,1482.0,H,NaN,NaN,I,1920,191475425,RSA5
3,D,O50,ROW 3 STY MASONRY,1,Single Family,Y,100.0,4.0,0.0,14.33,...,F,1433.0,2049.0,A,NaN,NaN,I,1920,191475425,RSA5
4,C,O50,ROW 3 STY MASONRY,1,Single Family,N,100.0,4.0,0.0,15.00,...,F,1500.0,1155.0,H,NaN,NaN,I,1920,191475425,RSA5


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581456 entries, 0 to 581455
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   basements                  325571 non-null  object 
 1   building_code              581456 non-null  object 
 2   building_code_description  581456 non-null  object 
 3   category_code              581456 non-null  int64  
 4   category_code_description  581456 non-null  object 
 5   central_air                285860 non-null  object 
 6   depth                      581454 non-null  float64
 7   exterior_condition         556191 non-null  float64
 8   fireplaces                 581435 non-null  float64
 9   frontage                   581454 non-null  float64
 10  fuel                       10900 non-null   object 
 11  garage_spaces              581435 non-null  float64
 12  garage_type                530522 non-null  object 
 13  general_construction       51

To ease the exploratory data analysis process, we grouped the columns based on its types and its connections with other columns.

1. Property Location:
    * Location, Street Designation
2. Classification of Property:
    * Building code, Building code description, Category code, Category code description
    * Unit
    * Zoning
    * Unfinished
    * Zip code
3. Property Specifications:
    * Basements, Garage type, Garage spaces
    * Central air, Fireplaces, Other building, Unfinished, Topography, View type
    * Depth, Frontage, Total area, Total livable area
    * Exterior condition, Interior condition, Quality grade, General construction
    * Fuel, Type heater
    * Number of bathrooms, Number of bedrooms, Number of rooms, Number stories, Year built 
    * Market value, Sale price, Sale date
4. Others:
    * Parcel number, Parcel shape
    * Recording date, registry number


# Cleaning and Initial EDA of Features

<strong> Category Code, Building Code and Zoning </strong>

In [10]:
df[['category_code','category_code_description']].dtypes

category_code                 int64
category_code_description    object
dtype: object

In [11]:
df['category_code'].value_counts()

1    461573
6     44612
2     42462
3     14396
4     14059
5      4354
Name: category_code, dtype: int64

In [12]:
df['category_code_description'].value_counts()

Single Family    461573
Vacant Land       44612
Multi Family      42462
Mixed Use         14396
Commercial        14059
Industrial         4354
Name: category_code_description, dtype: int64

<b> Building Code </b>

In [13]:
df[['building_code','building_code_description']].dtypes

building_code                object
building_code_description    object
dtype: object

In [14]:
df[['building_code','building_code_description']].nunique()

building_code                802
building_code_description    798
dtype: int64

<b> Zoning </b>


In [15]:
df['zoning'].dtypes

dtype('O')

In [16]:
df['zoning'].nunique()

43

In [17]:
df['zoning'] = df['zoning'].apply(lambda x: str(x).strip())
df['zoning'].unique()

array(['RSA5', 'ICMX', 'RM1', 'CMX1', 'I2', 'CMX2', 'RMX2', 'CMX25',
       'CA1', 'nan', 'CMX3', 'SPPOA', 'RM2', 'RSA3', 'CMX2.', 'CMX4',
       'RSA1', 'RSD1', 'IRMX', 'RMX3', 'CMX5', 'RM4', 'I1', 'RS3', 'RSA2',
       'RTA1', 'RSD3', 'RMX1', 'RM3', 'RSA', 'SPINS', 'RSD2', 'CA2', 'I3',
       'SPAIR', 'RSA4', '12', 'IP', 'SPSTA', 'SC', 'SPENT', '2002',
       'SPPOP', 'SP'], dtype=object)

In [18]:
df['zoning'].value_counts()

RSA5     288769
RM1      105876
RSA3      62750
CMX2      20772
RSD3      13180
RSA2      12602
RSA4      12188
CMX3       8053
RTA1       5766
RM2        5547
CMX1       5445
CMX5       4915
CMX25      4335
I2         4270
CMX4       4194
RM4        3814
RSD1       3270
ICMX       2993
RMX3       2276
RSA1       1294
RM3        1292
RMX1       1275
RSD2       1274
SPPOA       868
CA1         844
I1          725
IRMX        708
CA2         628
I3          433
SPINS       366
RMX2        271
CMX2.       211
nan         133
12           45
SPAIR        23
IP           23
SPSTA        16
RSA           3
SPENT         3
SPPOP         2
2002          1
RS3           1
SC            1
SP            1
Name: zoning, dtype: int64

<b> Dropping Anomalies </b>


In [19]:
index = df[df['zoning']=='2002'].index
df.drop(index=index, axis=0, inplace=True)

In [20]:
index = df[df['zoning']=='SP'].index
df.drop(index=index, axis=0, inplace=True)

In [21]:
index = df[df['zoning']=='RS3'].index
df.drop(index=index, axis=0, inplace=True)

In [22]:
index = df[df['zoning']=='SC'].index
df.drop(index=index, axis=0, inplace=True)

In [23]:
index = df[df['zoning']=='RSA'].index
df.drop(index=index, axis=0, inplace=True)

<b> Fixing Typos </b>

In [24]:
df['zoning'] = df['zoning'].apply(lambda x: 'I2' if x=='12' else x)

In [25]:
df['zoning'] = df['zoning'].apply(lambda x: 'CMX2' if x=='CMX2' else x)


In [26]:
df['zoning'] = df['zoning'].apply(lambda x: np.nan if x=='nan' else x)


In [27]:
df['zoning'].isna().sum()

133

<b> Dropping NAs </b>

In [28]:
index = df[df['zoning'].isna()].index
df.drop(index=index, axis=0, inplace=True)

In [29]:
df['zoning'].isna().sum()

0

In [30]:
df['new_zoning']=df['zoning'].apply(lambda x: 'Residential Single Family' if (x=='RSD1') | (x=='RSD2') | (x=='RSD3') 
                   | (x=='RSA1') | (x=='RSA2') | (x=='RSA3') | (x=='RSA4') | (x=='RSA5') else 
                   'Residential Two Family' if (x=='RTA1') else
                   'Residential Multi Family' if (x=='RM1') | (x=='RM2') | (x=='RM3') | (x=='RM4') else
                   'Mixed Used' if (x=='RMX1') | (x=='RMX2') | (x=='RMX3') | (x=='CMX1') | (x=='CMX2') 
                   | (x=='CMX25') | (x=='CMX3') | (x=='CMX4') | (x=='CMX5') | (x=='ICMX') | (x=='IRMX')else
                   'Commercial' if (x=='CA1') | (x=='CA2') else
                   'Industrial' if (x=='I1') | (x=='I2') | (x=='I3') | (x=='IP') else 
                   'Special Purpose Airport' if (x=='SPAIR') else 
                   'Special Purpose Institution' if (x=='SPINS') else 
                   'Special Purpose Entertainment' if (x=='SPENT') else 
                   'Special Purpose Stadium' if (x=='SPSTA') else 
                   'Special Purpose Open Space-Active' if (x=='SPPOA') else 
                   'Special Purpose Open Space-Passive' if (x=='SPPOP') else '0')

In [31]:
df['new_zoning'].value_counts()

Residential Single Family             395327
Residential Multi Family              116529
Mixed Used                             55237
Residential Two Family                  5766
Industrial                              5496
Commercial                              1472
Special Purpose Open Space-Active        868
Special Purpose Institution              366
0                                        211
Special Purpose Airport                   23
Special Purpose Stadium                   16
Special Purpose Entertainment              3
Special Purpose Open Space-Passive         2
Name: new_zoning, dtype: int64

<strong> Crosschecking Category and Building Code </strong>

In [32]:
df['new_zoning']

0         Residential Single Family
1         Residential Single Family
2         Residential Single Family
3         Residential Single Family
4         Residential Single Family
                    ...            
581451                   Mixed Used
581452                   Mixed Used
581453                   Mixed Used
581454                   Mixed Used
581455                   Mixed Used
Name: new_zoning, Length: 581316, dtype: object

In [33]:
pd.set_option('display.max_rows',802)
df.groupby('category_code_description')['building_code_description'].value_counts()

category_code_description  building_code_description 
Commercial                 STORE 1 STY MASONRY             1549
                           AUTO REPAIR SHOP MASONRY        1109
                           HSE WORSHIP ALL 2 STY MAS        740
                           COM CONDO 1 STY MASONRY          721
                           HSE WORSHIP ALL 1 STY MAS        552
                           STORE 2 STY MASONRY              462
                           STR/OFF 2 STY MASONRY            455
                           STR/OFF 1 STY MASONRY            434
                           MISC DAY CARE MASONRY            353
                           HSE WORSHIP ALL 3 STY MAS        327
                           OFF BLD N/COM W/PKG MASON        294
                           OFF BLD N/PKG N/COM MASON        273
                           SCHOOL 3 STY MASONRY             271
                           REST'RNT W/BAR MASONRY           261
                           STORE 3 STY MASONRY    

In [34]:
def checking(c):
    c = c.str.extractall('([a-zA-Z]+)')
    c.columns=['check']
    b = c.reset_index(drop=True)
    c = b['check'].value_counts()
    
    checking = {'checking':c.index, 'freq':c.values}
    checking = pd.DataFrame(checking)
    checking.index = checking['checking']
    checking.drop('checking',axis=1,inplace=True)
    checking.sort_values('freq',ascending=False,inplace=True)
    return checking

In [35]:
check_1=checking(df[df['category_code']==1]['building_code_description'])
check_2=checking(df[df['category_code']==2]['building_code_description'])
check_3=checking(df[df['category_code']==3]['building_code_description'])
check_4=checking(df[df['category_code']==4]['building_code_description'])
check_5=checking(df[df['category_code']==5]['building_code_description'])
check_6=checking(df[df['category_code']==6]['building_code_description'])

In [36]:
check_1.rename(columns={'freq':'Single Family'},inplace=True)
check_2.rename(columns={'freq':'Multi Family'},inplace=True)
check_3.rename(columns={'freq':'Mixed Use'},inplace=True)
check_4.rename(columns={'freq':'Commercial'},inplace=True)
check_5.rename(columns={'freq':'Industrial'},inplace=True)
check_6.rename(columns={'freq':'Vacant Land'},inplace=True)

In [37]:
check_count = pd.concat([check_1,check_2,check_3,check_4,check_5,check_6],axis=1)
check_count.fillna(0,inplace=True)

In [38]:
check_count

,Single Family,Multi Family,Mixed Use,Commercial,Industrial,Vacant Land
checking,,,,,,
STY,459237.0,37284.0,14396.0,7336.0,4.0,0.0
MASONRY,406197.0,5151.0,6867.0,9618.0,3966.0,0.0
ROW,332555.0,16173.0,7051.0,0.0,0.0,0.0
GAR,177091.0,0.0,0.0,369.0,0.0,0.0
B,145700.0,0.0,797.0,0.0,0.0,0.0
W,66418.0,0.0,6303.0,814.0,0.0,0.0
DET,56920.0,1008.0,218.0,0.0,0.0,0.0
D,51978.0,5318.0,440.0,0.0,0.0,0.0
S,38755.0,5318.0,440.0,5.0,0.0,0.0


By skimming two tables above, it seems that the category code was classified using building code description. But, since the data is a lot to check with, we will answer the question using a simple modeling.

In [39]:
text = df['building_code_description']
label = df['category_code_description']


In [40]:
text_train, text_test, y_train, y_test = train_test_split(text, label, stratify=label, random_state=2020)

In [41]:
vect = CountVectorizer().fit(text_train)

In [42]:
x_train = vect.transform(text_train)
x_test = vect.transform(text_test)

In [43]:
feature = vect.get_feature_names_out()

In [44]:
model = LogisticRegression(n_jobs=-1)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [45]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

   Commercial       0.98      1.00      0.99      3513
   Industrial       1.00      1.00      1.00      1088
    Mixed Use       1.00      0.98      0.99      3599
 Multi Family       1.00      1.00      1.00     10601
Single Family       1.00      1.00      1.00    115388
  Vacant Land       1.00      1.00      1.00     11140

     accuracy                           1.00    145329
    macro avg       1.00      1.00      1.00    145329
 weighted avg       1.00      1.00      1.00    145329



This model proves that building code description can specifically predict which category code the data belongs to. It's very specific that it proves that there are no misclassification of category code in the dataset.

This also confirms that category code is representing the type of the property, not the building. For example, logically, condominium (CONDO) should be classified into multi family if category code represents the building.

In [46]:
df[df['building_code_description'].str.contains('CONDO')].value_counts('category_code_description')

category_code_description
Single Family    34656
Commercial         932
Industrial         101
dtype: int64

Changing the Category Code Description to following.
* Resident
* Hotel and Apartments
* Store and Dwelling
*  Commercial
*  Industrial
*  Vacant Land

In [48]:
dupe = pd.read_csv('./Data/dupe.csv')
dupe.drop(columns=['Unnamed: 0'], inplace=True)
len(dupe)

3634

In [49]:
df_dupe = df[df['location'].isin(dupe['dupe'])]
len(df_dupe)

37692

In [50]:
len(df_dupe)/len(df)*100

6.483908923889932

We have 37692 (6.5% from total) data with duplicate 'location'.

# Crosschecking

In [52]:
df['unit'].dtypes

dtype('O')

In [54]:
df['unit'].nunique()

8761

In [55]:
len(df[df['unit'].notna()])

38717

In [56]:
len(df_dupe[df_dupe['unit'].notna()])/len(df[df['unit'].notna()])*100

96.19546969031691

Most of the data with filled 'unit' value were located in data with duplicate 'location' value (df_dupe).

In [57]:
check = df_dupe[['location','unit','sale_date']].sort_values(by=['location','unit'])
check[0:100]

,location,unit,sale_date
575439,1 ACADEMY CIR,00000P1,2009-08-03 00:00:00
570503,1 ACADEMY CIR,00000P2,2011-08-12 00:00:00
575440,1 ACADEMY CIR,00000P3,2016-04-22 00:00:00
575441,1 ACADEMY CIR,00000P4,2016-07-01 00:00:00
575442,1 ACADEMY CIR,00000P5,2007-01-25 00:00:00
575443,1 ACADEMY CIR,00000P6,2006-11-17 00:00:00
575444,1 ACADEMY CIR,00000P7,2009-09-29 00:00:00
575445,1 ACADEMY CIR,00000P8,2006-10-23 00:00:00
575446,1 ACADEMY CIR,00000P9,2006-10-25 00:00:00
573269,1 ACADEMY CIR,0000101,2019-06-17 00:00:00


In [59]:
location = []
for i in check['location'].unique():
    unit = []
    for j in check[check['location']==i]['unit']:
        if j not in unit:
            unit.append(j)
        elif j in unit:
            location.append(i)
location

['1100 S CHRIS COLUMBUS BLV',
 '1126-36 ARCH ST',
 '115 SOUTH ST',
 '1341 S CHRIS COLUMBUS BLV',
 '1401 S CHRIS COLUMBUS BLV',
 '1500 JOHN F KENNEDY BLVD',
 '1801 JOHN F KENNEDY BLVD',
 '1801 W HUNTING PARK AVE',
 '2906 S CHRIS COLUMBUS BLV',
 '3701 S CHRIS COLUMBUS BLV',
 '4101 S CHRIS COLUMBUS BLV',
 '4201 WALNUT ST',
 '700 WALNUT LN',
 '7901-03 FRANKFORD AVE',
 '8998 PINE RD',
 '948 N LAWRENCE ST']